In [1]:
%run /DemandSensing/_Main/RunAll

Archival Process Begin - the files(EUShipmentFinal,EUDSFinal,EUDPFinal) must exist before archival at PV_delivery_Laundry_Reporting()

In [3]:
FileExtention = '.parquet'
Filename = 'EUShipmentFinal.parquet'
Filenamenoext = 'EUShipmentFinal'
path = PV_delivery_Laundry_Reporting()
ArchivalProcessanyparqt(Filenamenoext,path)
dbutils.fs.rm(path+Filename , True)
Filename1 = 'EUDSFinal.parquet'
Filenamenoext1 = 'EUDSFinal'
ArchivalProcessanyparqt(Filenamenoext1,path)
dbutils.fs.rm(path+Filename1 , True)
Filename2 = 'EUDPFinal.parquet'
Filenamenoext2 = 'EUDPFinal'
ArchivalProcessanyparqt(Filenamenoext2,path)
dbutils.fs.rm(path+Filename2 , True)

Out[ 268 ]: True

In [4]:
%%script false
##Archive the final files and to create new file every run and maintain history for review
FileExtention = '.csv'
Filename = 'EUShipmentFinal.csv'
Filenamenoext = 'EUShipmentFinal'
path = PV_delivery_Laundry_Reporting()
ArchivalProcessanycsv(Filenamenoext,path)
dbutils.fs.rm(path+Filename , True)

In [5]:
%%script false
##Archive the final files and to create new file every run and maintain history for review
Filename1 = 'EUDSFinal.csv'
Filenamenoext1 = 'EUDSFinal'
ArchivalProcessanycsv(Filenamenoext1,path)
dbutils.fs.rm(path+Filename1 , True)


In [6]:
%%script false
##Archive the final files and to create new file every run and maintain history for review
Filename2 = 'EUDPFinal.csv'
Filenamenoext2 = 'EUDPFinal'
ArchivalProcessanycsv(Filenamenoext2,path)
dbutils.fs.rm(path+Filename2 , True)

Func helps LAG calculation - input the year.weeknumber format ie 2019.28 it returns the date 2019-07-09 Check the weekday of a Jan 1st of a year in this case 2019 which is a Tuesday so it must return a tuesday date for any weeknumber in the year 2019
LAG is difference in weeks between dates return by the func for the input - SDATE and VERSION in case for shipment similarly for DS and DP

In [8]:
%%script false
##Calculates the LAG as per SSIS Package 
##UDF slower but only way to mimic logic for LAG
import sys
import datetime
from time import strptime, strftime
from pyspark.sql.types import DateType, DoubleType, IntegerType, StringType

def _getWeekDetails(_weekNo, _Year, _weekStart):
    rslt = []
    janOne = strptime('%s-01-01' % _Year, '%Y-%m-%d')
    dayOfFirstWeek = ((7-int((strftime("%u",janOne)))+ int(_weekStart)) % 7)
    if dayOfFirstWeek == 0:
        dayOfFirstWeek = 7
    dateOfFirstWeek = strptime('%s-01-%s' % (_Year, dayOfFirstWeek), '%Y-%m-%d')
    dayOne = datetime.datetime( dateOfFirstWeek.tm_year, dateOfFirstWeek.tm_mon, dateOfFirstWeek.tm_mday )
    daysToGo = 7*(int(_weekNo)-1)
    lastDay = daysToGo+6
    dayX = dayOne + datetime.timedelta(days = daysToGo)
    #dayY = dayOne + datetime.timedelta(days = lastDay)
    resultDateX = strptime('%s-%s-%s' % (dayX.year, dayX.month, dayX.day), '%Y-%m-%d')
    #resultDateY = strptime('%s-%s-%s' % (dayY.year, dayY.month, dayY.day), '%Y-%m-%d')
    rslt.append(resultDateX)
    #rslt.append(resultDateY)
    return rslt

def get_first_week_of_date(strWeekNumYear):
    year = strWeekNumYear[0:4]
    weeknum = strWeekNumYear[5:7]
    startOfWeek = datetime.datetime(int(year), 1, 1, 23, 24, 55, 173504).weekday()

    WeekData = _getWeekDetails(weeknum, year, startOfWeek + 2)
    str1 = strftime("%Y-%m-%d", WeekData[0])
    return str1

#pyspark_get_first_week_of_date = udf(lambda z: get_first_week_of_date(z), StringType())
pyspark_get_first_week_of_date  = udf(get_first_week_of_date, StringType())

Load all csv for shipment,DP,DS and Item to be used for ETL

In [10]:

EUDP_FCST_Filenamewithext = '315741173_315741173-MDSL_eu-07-dp-forecast_1.0.csv'
EUDS_FCST_Filenamewithext = '315741173-MDSL_315741173_eu-07-mds-forecast_1.0.csv'
FCST_VERSION_APO_Filenamewithext = 'TableVersionsforDemandSensingReporting_v1.csv'
EUShipment_Filenamewithext = '315741173_315741173-MDSL_eu-07-shipment-history_1.0.csv'
EUItem_Filenamewithext = '315741173_315741173-MDSL_eu-07-item-master_1.0.csv'

EUDP_FCST_Filename = '315741173_315741173-MDSL_eu-07-dp-forecast_1.0'
EUDS_FCST_Filename = '315741173-MDSL_315741173_eu-07-mds-forecast_1.0'                    
FCST_VERSION_APO_Filename = 'FRCT_VERSION_LIST'#'TableVersionsforDemandSensingReporting_v1'
EUShipment_Filename = '315741173_315741173-MDSL_eu-07-shipment-history_1.0'
EUItem_Filename = '315741173_315741173-MDSL_eu-07-item-master_1.0'

# PV_FCST_VERSION_APO adl://hdf.azuredatalakestore.net/lab/bu/x/dsp/trusted/APO-Database/Table%20Versions%20for%20Demand%20Sensing%20Reporting_v1.csv
dbutils.fs.cp(PV_L1_CUR_FCST_APO()+EUDP_FCST_Filenamewithext,PV_delivery_Laundry_Reporting()+'archive/DP/'+EUDP_FCST_Filename+time.strftime("_%Y%m%d_%H%M%S")+'.csv',True) 
dbutils.fs.cp(PV_L10_RTFORDER_APO()+EUDS_FCST_Filenamewithext,PV_delivery_Laundry_Reporting()+'archive/DS/'+EUDS_FCST_Filename+time.strftime("_%Y%m%d_%H%M%S")+'.csv',True) 

EUDP_FCST_df = fileToDF_old(PV_L1_CUR_FCST_APO(),EUDP_FCST_Filename,'csv',delimiterSign='|',headerAvail="false")
EUDS_FCST_df = fileToDF_old(PV_L10_RTFORDER_APO(),EUDS_FCST_Filename,'csv',delimiterSign='|',headerAvail="false")
FCST_VERSION_APO = fileToDF_old(PV_FCST_VERSION_APO(),FCST_VERSION_APO_Filename,'csv',delimiterSign='|')
EUShipment_df = fileToDF_old(PV_L1_HIST_Shipment_FILE_Laundry(),EUShipment_Filename,'csv',delimiterSign='|',headerAvail="false")
EUItem_df = fileToDF_old(PV_L1_HIST_IM_FILE_Laundry(),EUItem_Filename,'csv',delimiterSign='|',headerAvail="false")



##Delete the Source files for DS and DP in SSIS- add logic

In [11]:
#ESL1DO07  ONLY SPAIN
#LOC	ITEM	CUST	CDATE	RDATE	SDATE	pdate	order_num	line_item_num	SQTY	UOM	RQTY	SalesArea	Region_Business
#S4Y1	2242113	2193409	20180727 081405	20180727	20180727		401379271	550	2.000	CON	2.000	HUB5DO05	EU_BEAUTY_CARE
##Prep the file column header for shipment
EUShipment_df1 = EUShipment_df
#.filter(EUShipment_df._c12 == 'ESL1DO07')
a = EUShipment_df1.alias("a")
EUShipment_df2 = a.select(a._c0.alias('LOC'),
a._c1.alias('ITEM'),a._c2.alias('CUST'),a._c3.alias('CDATE'),a._c4.alias('RDATE'),a._c5.alias('SDATE'),a._c6.alias('pdate'),a._c7.alias('order_num'),
a._c8.alias('line_item_num'),a._c9.alias('SQTY'),a._c10.alias('UOM'),a._c11.alias('RQTY'),a._c12.alias('SalesArea'),a._c13.alias('Region_Business'))
##.filter( (a._c1 == '2279129')  )
##EUShipment_df2.show(n=10)

#CDATE	DP_LOC	ITEM	SALESAREA	TOPAPOPARTNER	APOPARTNER	CUST_L2	FTYPE	FDATE	BUCKET	UOM	FQTY	SYSQTY	REGION_BUSINESS	TIMESTAMP
#20190305	S4V1	2427217	FIG1DO00		APOFIG1D00	OTHERS FINLAND LEV2	    1	20190415	              7	CON	0.816	0.827	EU_BEAUTY_CARE	2019-03-05 23:31:50.123
##Prep the file column header for DP

b = EUDP_FCST_df.alias("b")
EUDP_FCST_df1 = b.select(b._c0.alias('CDATE'),
b._c1.alias('LOC'),b._c2.alias('ITEM'),b._c3.alias('SALESAREA'),b._c4.alias('TOPAPOPARTNER'),b._c5.alias('APOPARTNER'),b._c6.alias('CUST_L2'),b._c7.alias('FTYPE'),b._c8.alias('FDATE'),b._c9.alias('BUCKET'),b._c10.alias('UOM'),b._c11.alias('FQTY'),b._c12.alias('SYSQTY'),b._c13.alias('Region_Business'))
##.filter((b._c3 == 'ESL1DO07') )
##& (b._c2 == '2279129' )
##EUDP_FCST_df1.show(n=10)

#LOC	ITEM	CHANNEL	FDATE	BUCKET	TOTAL_FCST	PROMO_FCST	UOM	REGION_BUSINESS	TIMESTAMP
#S4Y1	2148867	HUB5DO05-ALL-OTHERS	20180405	1	5.000	0.000	CON	EU_BEAUTY_CARE	2018-03-21 21:00:00.000
##Prep the file column header for DS

c = EUDS_FCST_df.alias("c")
EUDS_FCST_df1 = c.select(c._c0.alias('LOC'),c._c1.alias('ITEM'),c._c2.alias('SALESAREA'),c._c3.alias('FDATE'),c._c4.alias('BUCKET'),c._c5.alias('TOTAL_FCST'),c._c6.alias('PROMO_FCST'),c._c7.alias('UOM'),c._c8.alias('REGION_BUSINESS'))
##.filter((c._c2 == 'ESL1DO07'))
##   & (c._c1 == '2279129' )
##EUDS_FCST_df1.show(n=10)

c1 = EUDS_FCST_df1.alias("c1")

#ITEM	IDESCR	SIGRP	PUOM	IType1	IType2	IType3	IGroup1	IGroup2	IGroup3	ICat1	ICat2	ICat3	IFAM1	IFAM2	IFAM3	UOM1	UOM2	UOM3	CONV1	CONV2	CONV3	Region_Business
#1006099	ESTRELLA OXYGENO 0L5 STD 16UD		CON	WH1	ALL PURPOSE CLEANERS	ESTRELLA BLEACH	ESTRELLA OXI	500 ML	ACTIVE OXYGEN	STANDARD ARTICLE	DUMMY	8410020379581	072130677375A0GJGN			SHU	SMU	PAL	0.063	1.000	0.001	EU_LAUNDRY
i = EUItem_df.alias("i")
##DFToView(i,'i')
EUItem_df1 = i.select(i._c0.alias('ITEM'),i._c1.alias('IDESCR'),i._c2.alias('SIGRP'),i._c3.alias('PUOM'),i._c4.alias('IType1'),i._c5.alias('IType2'),i._c6.alias('IType3'),i._c7.alias('IGroup1'),
i._c8.alias('IGroup2'),i._c9.alias('IGroup3'),i._c10.alias('ICat1'),i._c11.alias('ICat2'),i._c12.alias('ICat3'),i._c13.alias('IFAM1'),i._c14.alias('IFAM2'),i._c15.alias('IFAM3'),i._c16.alias('UOM1'),i._c17.alias('UOM2'),i._c18.alias('UOM3'),i._c19.alias('CONV1'),i._c20.alias('CONV2'),i._c21.alias('CONV3'),i._c22.alias('Region_Business'))
#EUItem_df.show(n=10)
i1 = EUItem_df1.alias("i1")
DFToView(i1,'i1')

We have a static file holding first date of a week in a particular year in future in case of removing dependency we can enhance the code using UDF pyspark_get_first_week_of_date()

In [13]:
##Ready static data for missing versions
from pyspark.sql.functions import split, format_number, trim

q = FCST_VERSION_APO.alias("q")

#TBD
#q.withColumn('DAYOFWEEK',to_date(pyspark_get_first_week_of_date(q['F_WEEK']),"%y-%M-%d"))
#.withColumn('splitd',split(q1['VERSION'],'.')[0])
#.withColumn('check',format_number(q1.VERSION,2))
#f.regexp_replace(,"\\,","")
##.withColumn('DAYOFVERSIONFormat',to_date(q.DAYOFVERSION,"%y-%M-%d").cast('string'))
#TBD


FCST_VERSION_APO1 = q.withColumn('DAYOFWEEK',pyspark_get_first_week_of_date(f.trim(q['F_WEEK']).cast('string'))).withColumn('VERSIONFormat',regexp_replace(format_number(q.VERSION,2).cast('string'),",","")).withColumn('F_WEEKFormat',regexp_replace(format_number(q.F_WEEK,2).cast('string'),",","")).withColumn('DAYOFVERSIONFormat',pyspark_get_first_week_of_date(f.trim(regexp_replace(format_number(q.VERSION,2).cast('string'),",",""))))

#TBD#DFToView(FCST_VERSION_APO1,'FCST_VERSION_APO1')

#q.show(n=10)

q1 = FCST_VERSION_APO1.alias("q1")

#TBD#####to_date(q1['DAYOFVERSION'],"%y-%M-%d").cast('string').alias('DAYOFVERSIONFormat')

#Final column list
FCST_VERSION_APO2 = q1.select(q1.VERSIONFormat,q1.DAYOFVERSIONFormat,q1.F_WEEKFormat,q1.DAYOFWEEK)
###FCST_VERSION_APO2.where( (q1['VERSION'].cast('string').like("%2019.03%")) )
#DFToView(FCST_VERSION_APO2,'FCST_VERSION_APO2')
##FCST_VERSION_APO2.show(n=10)

#FilenameStage = 'FCST_VERSION_APO2STG'
#FileExtention = '.csv'
#Filename = 'FCST_VERSION_APO2.csv'

#dfToSingleFile(FCST_VERSION_APO2, PV_delivery_Laundry_Reporting()+FilenameStage+FileExtention,'csv',delimiterSign='|')


#DFToView(FCST_VERSION_APO2,'FCST_VERSION_APO2')
s = FCST_VERSION_APO2.alias("s") 


Transforming Shipment file for ETL

In [15]:
##ShipmentEU
a2 = EUShipment_df2.alias("a2")
calendar.setfirstweekday(calendar.MONDAY)
EUShipment_df3 = a2.withColumn('SDATEFormat',f.concat(f.year(to_date(a2['SDATE'].cast('string'), "yyyyMMdd")),lit('.'),lpad(f.weekofyear(to_date(a2['SDATE'].cast('string'), "yyyyMMdd")),2,'0')))
a33 = EUShipment_df3.alias("a33")
EUShipment_df4 = a33.withColumn('DAYOFWEEK',pyspark_get_first_week_of_date(trim(a33['SDATEFormat'])))
a3 = EUShipment_df4.alias("a3")
DFToView(EUShipment_df4,'EUShipment_df4')
#a3.show(n=10)

IDH_SHIPMENTS_FRCST_STG01_EU = a3.select(a3.ITEM,a3.SalesArea,a3.UOM,a3.SDATEFormat,a3.DAYOFWEEK,a3.SQTY).sort(a3.SDATEFormat).groupBy(a3.ITEM,a3.SalesArea,a3.UOM,a3.SDATEFormat,a3.DAYOFWEEK).agg(f.sum('SQTY').alias('SQTYShip')) ##REMOVE FILTER MX DNF  & (a3.SDATEFormat == '2019.31') .filter( (a3.ITEM == '2279129')  )
a4 = IDH_SHIPMENTS_FRCST_STG01_EU.alias("a4")

EUMissingSVersion2 = s.join(a4,[a4.SDATEFormat == s.F_WEEKFormat],'inner').select(a4.ITEM,a4.SalesArea,a4.SDATEFormat,a4.DAYOFWEEK,s.DAYOFVERSIONFormat,a4.UOM,s.VERSIONFormat)
EUMissingSVersion2.dropDuplicates()
DFToView(EUMissingSVersion2,'MissingSVersion2')
#EUMissingSVersion2.show(n=10)
##DFToView(EUMissingSVersion2,'EUMissingSVersion2')
a5 = EUMissingSVersion2.alias("a5")


EUMissingSVersion1 = a4.join(a5,[a4.ITEM == a5.ITEM,a4.SDATEFormat == a5.SDATEFormat,a4.SalesArea == a5.SalesArea],'right_outer').select(a5.SDATEFormat,a5.ITEM,a5.SalesArea,a5.VERSIONFormat.alias('VERSIONS'),a5.DAYOFVERSIONFormat,a5.DAYOFWEEK,a4.UOM,a5.VERSIONFormat,f.when(a4.SQTYShip.isNull(),lit(0)).otherwise(a4.SQTYShip).alias('SQTYShipTotal'),f.concat(lit('LagW_'),lpad(ceil(f.abs(datediff(a5.DAYOFVERSIONFormat,a5.DAYOFWEEK))/7),2,'0')).alias('LAGS')).sort(a5.SDATEFormat)
##DFToView(EUMissingSVersion1,'MissingSVersion')
##EUMissingSVersion1.show(n=10)


FilenameStage = 'EUShipmentFinalSTG'
FileExtention = '.csv'
Filename = 'EUShipmentFinal.csv'
Filenamenoext = 'EUShipmentFinal'

#dfToSingleFile(EUMissingSVersion1, PV_delivery_Laundry_Reporting()+Filenamenoext+FileExtention,'csv',delimiterSign='|')
#EUShipmentFinalcsv = fileToDF_old(PV_delivery_Laundry_Reporting(),Filenamenoext,'csv',delimiterSign='|')
dfToSingleFileFalse(EUMissingSVersion1, PV_delivery_Laundry_Reporting()+Filenamenoext+'.parquet','PARQUET',delimiterSign='|')

EUShipmentFinalcsv = fileToDF_old(PV_delivery_Laundry_Reporting(),Filenamenoext,'parquet',delimiterSign='|')
#DFToView(EUShipmentFinalcsv,'EUShipmentFinalcsv')

In [16]:
%%script false
def dfToMultipleFile(df, fpath, extention, delimiterSign = ';', writeMode = 'overwrite'):
  
  myTemp = '.parquet'
  
  #df = df.repartition(1)
  if extention.upper() == 'PARQUET' :
    df.write.mode(writeMode).option("maxRecordsPerFile", 20000).option("delimiter",delimiterSign).option("header", "false").parquet(fpath + myTemp)
  
  
  
 #fname = [f.path for f in dbutils.fs.ls(fpath + myTemp) if f.path.endswith('.'+ extention.lower())][0]
  ##dbutils.fs.mv(fname, fpath)
  #dbutils.fs.rm(fpath + myTemp, recurse=True) 

In [17]:
##dfToMultipleFile(EUMissingSVersion1, PV_delivery_Laundry_Reporting()+Filenamenoext,'PARQUET',delimiterSign='|')

Transforming DS and DP file for ETL for EU filter criteria on a Friday of every week

In [19]:
###DSEU Logic


EUDS_FCST_df2 = c1.withColumn('SALESAREAFormat',substring(trim(c1.SALESAREA),1,8)).withColumn('FDATEFormat',f.concat(f.year(to_date(c1['FDATE'].cast('string'), "yyyyMMdd")),lit('.'),lpad(f.weekofyear(to_date(c1['FDATE'].cast('string'), "yyyyMMdd")),2,'0'))).withColumn('VERSION1',lit('20190816')).withColumn('VERSION',date_format(f.current_date(), "yyyyMMdd")).withColumn('VERSIONFormat1',f.concat(f.year(to_date(lit('20190816'), "yyyyMMdd")),lit('.'),lpad(f.weekofyear(to_date(lit('20190816'), "yyyyMMdd")),2,'0'))).withColumn('VERSIONFormat',f.concat(f.year(f.current_date()),lit('.'),lpad(f.weekofyear(f.current_date()),2,'0')))

c2 = EUDS_FCST_df2.alias("c2")
#DFToView(c2,'c2')


EUDS_FCST_df3 = c2.withColumn('DAYOFVERSION',pyspark_get_first_week_of_date(trim(c2['VERSIONFormat1']))).withColumn('DAYOFWEEK',pyspark_get_first_week_of_date(trim(c2['FDATEFormat'])))

c3 = EUDS_FCST_df3.alias("c3")
#c3.show(n=10)
DFToView(c3,'c3')
EUDS_FCST_df4 = c3.withColumn('LAGFinal',f.concat(lit('LagW_'),lpad(ceil(f.abs(datediff(c3.DAYOFVERSION,c3.DAYOFWEEK))/7),2,'0')))

c4 = EUDS_FCST_df4.alias("c4")
##DFToView(c4,'c4')
#c4.show(n=10)

##MX change hardcoded value edit VERSION1 to VERSION
IDH_DS_FRCST_STG01_EU = c4.select (c4.ITEM,c4.SALESAREAFormat,c4.UOM,c4.VERSIONFormat1,c4.FDATEFormat,c4.LAGFinal,c4.DAYOFWEEK,c4.TOTAL_FCST).filter(( dayofweek(to_date(c4['VERSION1'], "yyyyMMdd")) == 6)).sort(c4.FDATE,c4.LAGFinal).groupBy(c4.ITEM,c4.SALESAREAFormat,c4.UOM,c4.VERSIONFormat1,c4.FDATEFormat,c4.LAGFinal,c4.DAYOFWEEK).agg(f.sum('TOTAL_FCST').alias('TOTAL_FCSTDS'))

c5 = IDH_DS_FRCST_STG01_EU.alias("c5")
#c5.show()
#DFToView(IDH_DS_FRCST_STG01_EU,'IDH_DS_FRCST_STG01_EU')

EUMissingDSVersion2 = s.join(c5,[c5.FDATEFormat == s.F_WEEKFormat, c5.LAGFinal <= 'LagW_12'],'inner').select(c5.ITEM,c5.SALESAREAFormat,c5.DAYOFWEEK,s.DAYOFVERSIONFormat,c5.UOM,s.VERSIONFormat,c5.FDATEFormat)
EUMissingDSVersion2.dropDuplicates()
#DFToView(EUMissingDSVersion2,'EUMissingDSVersion2')

c6 = EUMissingDSVersion2.alias("c6")
##c6.show(n=10)

EUMissingDSVersion1 =  c6.join(c5,[c6.ITEM == c5.ITEM,c6.SALESAREAFormat == c5.SALESAREAFormat,c6.DAYOFWEEK == c5.DAYOFWEEK,c6.VERSIONFormat == c5.VERSIONFormat1],'right_outer').select(c5.FDATEFormat,c5.ITEM,c5.SALESAREAFormat,c5.VERSIONFormat1.alias('VERSIONDS'),c5.DAYOFWEEK,c5.UOM,c6.VERSIONFormat,f.when(c5.TOTAL_FCSTDS.isNull(),lit(0)).otherwise(c5.TOTAL_FCSTDS).alias('FQTYDSTotal'),f.concat(lit('LagW_'),lpad(ceil(f.abs(datediff(c6.DAYOFVERSIONFormat,c5.DAYOFWEEK))/7),2,'0')).alias('LAGDS')).sort(c5.FDATEFormat)

##DFToView(EUMissingDSVersion1,'EUMissingDSVersion')
##EUMissingDSVersion1.show(n=10)
FilenameStage = 'EUDSFinalSTG'
FileExtention = '.csv'
Filename = 'EUDSFinal.csv'
Filenamenoext = 'EUDSFinal'

#dfToSingleFile(EUMissingDSVersion1, PV_delivery_Laundry_Reporting()+Filenamenoext+FileExtention,'csv',delimiterSign='|')
#EUDSFinalcsv = fileToDF_old(PV_delivery_Laundry_Reporting(),Filenamenoext,'csv',delimiterSign='|')
dfToSingleFileFalse(EUMissingDSVersion1, PV_delivery_Laundry_Reporting()+Filenamenoext+'.parquet','PARQUET',delimiterSign='|')
EUDSFinalcsv = fileToDF_old(PV_delivery_Laundry_Reporting(),Filenamenoext,'parquet',delimiterSign='|')

DFToView(EUDSFinalcsv,'EUDSFinalcsv')

In [20]:
##DP Logic
calendar.setfirstweekday(calendar.MONDAY)
b1 = EUDP_FCST_df1.alias("b1")

##.withColumn('FQTYIntTyp',regexp_replace(a['FQTY'].cast('string'),'\\.','').cast(IntegerType()))  as per APO team do not remove dot
EUDP_FCST_df2 = b1.withColumn('VERSION',f.concat(f.year(to_date(b1['CDATE'].cast('string'), "yyyyMMdd")),lit('.'),lpad(f.weekofyear(to_date(b1['CDATE'].cast('string'), "yyyyMMdd")),2,'0'))) \
  .withColumn('DP_WEEKFDATE',f.concat(f.year(to_date(b1['FDATE'].cast('string'), "yyyyMMdd")),lit('.'),lpad(f.weekofyear(to_date(b1['FDATE'].cast('string'), "yyyyMMdd")),2,'0'))) \
 .withColumn('LAG',f.concat(lit('LagW_'),lpad(ceil(f.abs(datediff(to_date(b1['CDATE'].cast('string'), "yyyyMMdd"),to_date(b1['FDATE'].cast('string'), "yyyyMMdd")))/7),2,'0'))).where(( dayofweek(to_date(b1['CDATE'].cast('string'), "yyyyMMdd")) == 6 )  ) 
##MX Remove ITEM Filter after DEV DONOTFORGET

b2 = EUDP_FCST_df2.alias("b2")
#b2.show(n=10)

EUDP_FCST_df3 = b2.withColumn('DAYOFVERSION',pyspark_get_first_week_of_date(trim(b2['VERSION']))).withColumn('DAYOFWEEK',pyspark_get_first_week_of_date(trim(b2['DP_WEEKFDATE'])))

b3 = EUDP_FCST_df3.alias("b3")
##DFToView(b3,'b3')
#b3.show(n=10)

EUDP_FCST_df4 = b3.withColumn('LAGFinal',f.concat(lit('LagW_'),lpad(ceil(f.abs(datediff(b3.DAYOFVERSION,b3.DAYOFWEEK))/7),2,'0')))
b4 = EUDP_FCST_df4.alias("b4")
##DFToView(b4,'b4')
#b4.show(n=10)

IDH_DP_FRCST_STG01_EU = b4.select(b4.ITEM,b4.SALESAREA,b4.UOM,b4.VERSION,b4.DP_WEEKFDATE,b4.LAGFinal,b4.FQTY,b4.DAYOFWEEK).sort(b4.LAGFinal,b4.DP_WEEKFDATE).groupBy(b4.ITEM,b4.SALESAREA,b4.UOM,b4.VERSION,b4.DP_WEEKFDATE,b4.LAGFinal,b4.DAYOFWEEK).agg(f.sum('FQTY').alias('FQTYSUM')) 

b5 = IDH_DP_FRCST_STG01_EU.alias("b5")
##insert here 
EUMissingDPVersion2 = s.join(b5,[b5.DP_WEEKFDATE == s.F_WEEKFormat, b5.LAGFinal <= 'LagW_12'],'inner').select(b5.ITEM,b5.SALESAREA,b5.DAYOFWEEK,s.DAYOFVERSIONFormat,b5.UOM,s.VERSIONFormat,b5.DP_WEEKFDATE)
EUMissingDPVersion2.dropDuplicates()
##DFToView(EUMissingDPVersion2,'EUMissingDPVersion2')

b6 = EUMissingDPVersion2.alias("b6")
#b6.show(n=10)
EUMissingDPVersion1 = b5.join(b6,[b5.ITEM == b6.ITEM,b5.SALESAREA == b6.SALESAREA,b5.DAYOFWEEK == b6.DAYOFWEEK,b5.VERSION == b6.VERSIONFormat],'right_outer').select(b6.DP_WEEKFDATE,b6.ITEM,b6.SALESAREA,b6.VERSIONFormat.alias('VERSION'),b6.DAYOFWEEK,b5.UOM,f.when(b5.FQTYSUM.isNull(),lit(0)).otherwise(b5.FQTYSUM).alias('FQTYTotal'),f.concat(lit('LagW_'),lpad(ceil(f.abs(datediff(b6.DAYOFVERSIONFormat,b6.DAYOFWEEK))/7),2,'0')).alias('LAGV1')).sort(b5.DP_WEEKFDATE)

 
DFToView(EUMissingDPVersion1,'EUMissingDPVersion')
EUMissingDPVersion1.show(n=10)

FilenameStage = 'EUDPFinalSTG'
FileExtention = '.csv'
Filename = 'EUDPFinal.csv'
Filenamenoext = 'EUDPFinal'

#dfToSingleFile(EUMissingDPVersion1, PV_delivery_Laundry_Reporting()+Filenamenoext+FileExtention,'csv',delimiterSign='|')
#EUDPFinalcsv = fileToDF_old(PV_delivery_Laundry_Reporting(),Filenamenoext,'csv',delimiterSign='|')
dfToSingleFileFalse(EUMissingDPVersion1, PV_delivery_Laundry_Reporting()+Filenamenoext+'.parquet','PARQUET',delimiterSign='|')
EUDPFinalcsv = fileToDF_old(PV_delivery_Laundry_Reporting(),Filenamenoext,'parquet',delimiterSign='|')
DFToView(EUDPFinalcsv,'EUDPFinalcsv')




+------------+----+---------+-------+---------+----+---------+-----+
DP_WEEKFDATE|ITEM|SALESAREA|VERSION|DAYOFWEEK| UOM|FQTYTotal|LAGV1|
+------------+----+---------+-------+---------+----+---------+-----+
 null|null| null|2019.29| null|null| 0.0| null|
 null|null| null|2019.35| null|null| 0.0| null|
 null|null| null|2019.42| null|null| 0.0| null|
 null|null| null|2019.37| null|null| 0.0| null|
 null|null| null|2019.23| null|null| 0.0| null|
 null|null| null|2019.37| null|null| 0.0| null|
 null|null| null|2019.31| null|null| 0.0| null|
 null|null| null|2019.28| null|null| 0.0| null|
 null|null| null|2019.31| null|null| 0.0| null|
 null|null| null|2019.28| null|null| 0.0| null|
+------------+----+---------+-------+---------+----+---------+-----+
only showing top 10 rows

The dataframes on the final files for shipment,DP and DS is combined to calculate Horizon cumulative totals and output

In [22]:
ds = EUDSFinalcsv.alias("ds")
dp = EUDPFinalcsv.alias("dp")
VW_EAN_DS_DP_AGG = dp.join(ds,[dp.SALESAREA == ds.SALESAREAFormat,dp.ITEM == ds.ITEM,dp.VERSION == ds.VERSIONDS,dp.DP_WEEKFDATE == ds.FDATEFormat],'full').select(f.when(dp.ITEM.isNull(),ds.ITEM).otherwise(dp.ITEM).alias('ITEM'),f.when(dp.SALESAREA.isNull(),ds.SALESAREAFormat).otherwise(dp.SALESAREA).alias('SALESAREA'),f.when(dp.UOM.isNull(),ds.UOM).otherwise(dp.UOM).alias('UOM'),f.when(dp.DP_WEEKFDATE.isNull(),ds.FDATEFormat).otherwise(dp.DP_WEEKFDATE).alias('WEEK'),f.when(dp.VERSION.isNull(),ds.VERSIONFormat.cast(IntegerType())).otherwise(dp.VERSION).alias('VERSION'),f.when(dp.LAGV1.isNull(),ds.LAGDS).otherwise(dp.LAGV1).alias('LAG'),f.when(dp.FQTYTotal.isNull(),lit(0)).otherwise(dp.FQTYTotal).alias('DPFQTY'),f.when(ds.FQTYDSTotal.isNull(),lit(0)).otherwise(ds.FQTYDSTotal).alias('DSFQTY'))

DFToView(VW_EAN_DS_DP_AGG,'VW_EAN_DS_DP_AGG')
#VW_EAN_DS_DP_AGG.show(n=10)
##,f.when(sp.VERSION.isNull(),s.VERSIONS).otherwise(sp.VERSION).alias('VERSION')
sp = VW_EAN_DS_DP_AGG.alias("sp")
DFToView(sp,'sp')
s = EUShipmentFinalcsv.alias("s")
DFToView(s,'s')

In [23]:
##Horizon and Cumulative Totals
ds = EUDSFinalcsv.alias("ds")
dp = EUDPFinalcsv.alias("dp")
VW_EAN_DS_DP_AGG = dp.join(ds,[dp.SALESAREA == ds.SALESAREAFormat,dp.ITEM == ds.ITEM,dp.VERSION == ds.VERSIONDS,dp.DP_WEEKFDATE == ds.FDATEFormat],'full').select(f.when(dp.ITEM.isNull(),ds.ITEM).otherwise(dp.ITEM).alias('ITEM'),f.when(dp.SALESAREA.isNull(),ds.SALESAREAFormat).otherwise(dp.SALESAREA).alias('SALESAREA'),f.when(dp.UOM.isNull(),ds.UOM).otherwise(dp.UOM).alias('UOM'),f.when(dp.DP_WEEKFDATE.isNull(),ds.FDATEFormat).otherwise(dp.DP_WEEKFDATE).alias('WEEK'),f.when(dp.VERSION.isNull(),ds.VERSIONFormat.cast(DoubleType())).otherwise(dp.VERSION.cast(DoubleType())).alias('VERSION'),f.when(dp.LAGV1.isNull(),ds.LAGDS).otherwise(dp.LAGV1).alias('LAG'),f.when(dp.FQTYTotal.isNull(),lit(0)).otherwise(dp.FQTYTotal).alias('DPFQTY'),f.when(ds.FQTYDSTotal.isNull(),lit(0)).otherwise(ds.FQTYDSTotal).alias('DSFQTY'))

DFToView(VW_EAN_DS_DP_AGG,'VW_EAN_DS_DP_AGG')
#VW_EAN_DS_DP_AGG.show(n=10)
##,f.when(sp.VERSION.isNull(),s.VERSIONS).otherwise(sp.VERSION).alias('VERSION')
sp = VW_EAN_DS_DP_AGG.alias("sp")
DFToView(sp,'sp')
s = EUShipmentFinalcsv.alias("s")
DFToView(s,'s')

In [24]:


myQuery=(
  """
 --%sql
SELECT DISTINCT IFNULL(sp.ITEM, s.ITEM) AS ITEM
      ,IFNULL(sp.SALESAREA,s.SALESAREA) AS SALESAREA
      ,IFNULL(sp.UOM,s.UOM) AS UOM
      ,CONCAT(LEFT(IFNULL(sp.VERSION,s.VERSIONS),4),'.',RIGHT(CONCAT('0',CAST((CAST(RIGHT(format_number(IFNULL(sp.VERSION,cast(s.VERSIONS as double)), 2),2) AS INT)+1) AS varchar(7))),2)) AS VERSION    
      ,IFNULL(sp.WEEK,s.SDATEFormat) AS WEEK
	  ,IFNULL(sp.LAG,s.LAGS) AS LAG
      ,IFNULL(sp.DPFQTY,0) AS DPFQTY
      ,IFNULL(sp.DSFQTY,0) AS DSFQTY
	  ,IFNULL(s.SQTYShipTotal,0) AS SQTY
      ,CASE WHEN s.SQTYShipTotal = 0 THEN sp.DPFQTY ELSE ABS(IFNULL(s.SQTYShipTotal,0) - IFNULL(sp.DPFQTY,0)) END AS ABS_DELT_SHIP_DP 
      ,CASE WHEN s.SQTYShipTotal = 0 THEN sp.DSFQTY ELSE ABS(IFNULL(s.SQTYShipTotal,0) - IFNULL(sp.DSFQTY,0)) END AS ABS_DELT_SHIP_DS 
  FROM   sp  FULL OUTER JOIN s    ON sp.ITEM = s.ITEM AND sp.VERSION = s.VERSIONS AND sp.SALESAREA = s.SalesArea AND sp.WEEK = s.SDATEFormat
     WHERE  
     IFNULL(sp.WEEK,s.SDATEFormat) < CONCAT (YEAR(NOW()),'.',RIGHT(CONCAT('0' ,CAST(WEEKOFYEAR(NOW()) AS VARCHAR(2))),2))
order by VERSION,WEEK,LAG
"""
)
##TBD  --s.ITEM = '2302896' 
ConsWeekIDH = executeSQL(myQuery,viewName='ConsWeekIDH')
fc = ConsWeekIDH.alias("fc")
ConsWeekIDH1 = ConsWeekIDH.withColumn('VERSIONR',f.when(fc.VERSION == '2018.53',lit('2019.01')).otherwise(fc.VERSION)).withColumn('Horizon',f.when(fc.LAG =='LagW_01',lit('H07')).when(fc.LAG =='LagW_02',lit('H014')).when(fc.LAG =='LagW_03',lit('H021')).when(fc.LAG =='LagW_04',lit('H028')).when(fc.LAG =='LagW_05',lit('H035')).when(fc.LAG =='LagW_06',lit('H042')).when(fc.LAG =='LagW_07',lit('H049')).when(fc.LAG =='LagW_08',lit('H056')).otherwise(lit('')))

fc1 = ConsWeekIDH1.alias("fc1")
DFToView(fc1,'fc1')

ConsWeekIDH2 = fc1.select(fc1.VERSIONR,fc1.WEEK,fc1.SALESAREA,fc1.ITEM,fc1.UOM,fc1.DPFQTY,fc1.DSFQTY,fc1.SQTY,fc1.LAG,fc1.Horizon,fc1.ABS_DELT_SHIP_DP,fc1.ABS_DELT_SHIP_DS).where((fc1.LAG.isin(['LagW_01','LagW_02','LagW_03','LagW_04','LagW_05','LagW_06','LagW_07','LagW_08'])) & (fc1.VERSIONR >='2018.10') )
fc2 = ConsWeekIDH2.alias("fc2")
DFToView(fc2,'fc2')

ConsWeekIDH3 = fc2.withColumn('DP_FRCST_H',f.sum(fc2.DPFQTY).over(Window.partitionBy(['SALESAREA','ITEM','VERSIONR']).orderBy(f.col('WEEK'),f.col('LAG')))).withColumn('DS_FRCST_H',f.sum(fc2.DSFQTY).over(Window.partitionBy(['SALESAREA','ITEM','VERSIONR']).orderBy(f.col('WEEK'),f.col('LAG')))).withColumn('SQTY_H',f.sum(fc2.SQTY).over(Window.partitionBy(['SALESAREA','ITEM','VERSIONR']).orderBy(f.col('WEEK'),f.col('LAG'))))
fc3 = ConsWeekIDH3.alias("fc3")
DFToView(fc3,'fc3')

ConsWeekIDH4 = fc3.withColumn('DAYOFVERSION',pyspark_get_first_week_of_date(trim(fc3['VERSIONR']))).withColumn('DAYOFWEEK',pyspark_get_first_week_of_date(trim(fc3['WEEK'])))
fc4 = ConsWeekIDH4.alias("fc4")
DFToView(fc4,'fc4')

##f.concat(f.when(fc4.ITEM.isNull(),i1.ITEM),lit('::'),f.when(i1.IDESCR.isNull(),''))
CONS_WEEK_IDH_DAY_HORIZON_EU = fc4.join(i1,[fc4.ITEM == i1.ITEM],'left_outer').select(fc4.ITEM,f.concat(f.when(fc4.ITEM.isNull(),i1.ITEM).otherwise(fc4.ITEM),lit('::'),f.when(i1.IDESCR.isNull(),'').otherwise(i1.IDESCR)).alias('ITEM_DESCRIPTION'),fc4.VERSIONR.alias('VERSION'),fc4.WEEK,fc4.DAYOFVERSION,fc4.DAYOFWEEK,fc4.SALESAREA,fc4.LAG,fc4.Horizon,fc4.DPFQTY,fc4.DP_FRCST_H,fc4.DSFQTY,fc4.DS_FRCST_H,fc4.SQTY,fc4.SQTY_H,fc4.ABS_DELT_SHIP_DP,i1.Region_Business,abs(fc4.SQTY_H - fc4.DP_FRCST_H).alias('ABS_DELT_SHIP_DP_H'),fc4.ABS_DELT_SHIP_DS,abs(fc4.SQTY_H - fc4.DS_FRCST_H).alias('ABS_DELT_SHIP_DS_H'))
fc5 = CONS_WEEK_IDH_DAY_HORIZON_EU.alias("fc5")
DFToView(fc5,'fc5')  

#dfToSingleFileFalse(CONS_WEEK_IDH_DAY_HORIZON_EU,PV_delivery_Laundry_Reporting()+'CONS_WEEK_IDH_DAY_HORIZON_EU.parquet','PARQUET',delimiterSign='|')
#dfToSingleFile(CONS_WEEK_IDH_DAY_HORIZON_EU,PV_delivery_Laundry_Reporting()+'CONS_WEEK_IDH_DAY_HORIZON_EU.csv','csv',delimiterSign='|')
dfToMultipleFile(CONS_WEEK_IDH_DAY_HORIZON_EU,PV_delivery_Laundry_Reporting()+'CONS_WEEK_IDH_DAY_HORIZON_EU','PARQUET',delimiterSign='|')

myQuery=(
  """
--%sql
SELECT i1.ICAT3 AS EAN
,fc5.ITEM
,fc5.VERSION
,fc5.WEEK
,fc5.DAYOFVERSION
,fc5.DAYOFWEEK
,fc5.SALESAREA
,fc5.LAG
,fc5.Horizon
,SUM(fc5.DPFQTY) AS DP_FRCST
,SUM(fc5.DP_FRCST_H) AS DP_FRCST_H
,SUM(fc5.DSFQTY) AS DS_FRCST
,SUM(fc5.DS_FRCST_H) AS DS_FRCST_H
,SUM(fc5.SQTY) AS SQTY
,SUM(fc5.SQTY_H) AS SQTY_H
,fc5.Region_Business
FROM fc5 LEFT OUTER JOIN i1 ON fc5.ITEM = i1.ITEM AND fc5.Region_Business = i1.Region_Business
group by i1.ICAT3 ,fc5.ITEM ,fc5.VERSION ,fc5.WEEK ,fc5.DAYOFVERSION ,fc5.DAYOFWEEK ,fc5.SALESAREA ,fc5.LAG ,fc5.Horizon ,fc5.Region_Business

"""
)
##TBD  --s.ITEM = '2302896' 
EAN_PRE_DAY_HORIZON_EU = executeSQL(myQuery,viewName='EAN_PRE_DAY_HORIZON_EU')
fc6 = EAN_PRE_DAY_HORIZON_EU.alias("fc6")
DFToView(fc6,'fc6')
#CONS_WEEK_EAN_DAY_HORIZON_EU = fc6.select(fc6.EAN,fc6.VERSION,fc6.DAYOFVERSION,fc6.WEEK,fc6.DAYOFWEEK,fc6.SALESAREA,fc6.LAG,fc6.Horizon,fc6.DP_FRCST,fc6.DP_FRCST_H,fc6.DS_FRCST,fc6.DS_FRCST_H,fc6.SQTY,fc6.SQTY_H,fc6.Region_Business,abs(fc6.SQTY - fc6.DP_FRCST).alias('ABS_DELT_SHIP_DP'),abs(fc6.SQTY - fc6.DS_FRCST).alias('ABS_DELT_SHIP_DS'),abs(fc6.SQTY_H - fc6.DP_FRCST_H).alias('ABS_DELT_SHIP_DP_H'),abs(fc6.SQTY_H - fc6.DS_FRCST_H).alias('ABS_DELT_SHIP_DS_H'))


CONS_WEEK_EAN_DAY_HORIZON_EU = fc6.select(fc6.EAN,fc6.VERSION,fc6.DAYOFVERSION,fc6.WEEK,fc6.DAYOFWEEK,fc6.SALESAREA,fc6.LAG,fc6.Horizon,fc6.DP_FRCST,fc6.DP_FRCST_H,fc6.DS_FRCST,fc6.DS_FRCST_H,fc6.SQTY,fc6.SQTY_H,fc6.Region_Business,abs(fc6.SQTY - fc6.DP_FRCST).alias('ABS_DELT_SHIP_D'),abs(fc6.SQTY - fc6.DS_FRCST).alias('ABS_DELT_SHIP_DS'),abs(fc6.SQTY_H - fc6.DP_FRCST_H).alias('ABS_DELT_SHIP_DP_H'),abs(fc6.SQTY_H - fc6.DS_FRCST_H).alias('ABS_DELT_SHIP_DS_H'))
fc7 = CONS_WEEK_EAN_DAY_HORIZON_EU.alias("fc7")
#fc7.withColumn('DateInserted',f.current_date())
#DFToView(fc7,'fc7')
#dfToSingleFileFalse(CONS_WEEK_EAN_DAY_HORIZON_EU, PV_delivery_Laundry_Reporting()+'CONS_WEEK_EAN_DAY_HORIZON_EU.parquet','PARQUET',delimiterSign='|')
#dfToSingleFile(CONS_WEEK_EAN_DAY_HORIZON_EU, PV_delivery_Laundry_Reporting()+'CONS_WEEK_EAN_DAY_HORIZON_EU.csv','csv',delimiterSign='|')
dfToMultipleFile(CONS_WEEK_EAN_DAY_HORIZON_EU, PV_delivery_Laundry_Reporting()+'CONS_WEEK_EAN_DAY_HORIZON_EU','PARQUET',delimiterSign='|')

                                                                                                        

Aggregate Quick Check for dev

In [26]:
EUIDH_HORIZON_REPORT_AGGR =fc5.select(fc5.SALESAREA,fc5.VERSION,fc5.WEEK,fc5.LAG,fc5.SQTY,fc5.DPFQTY,fc5.DSFQTY).sort(fc5.VERSION,fc5.WEEK,fc5.LAG).sort(fc5.SALESAREA,fc5.VERSION,fc5.WEEK,fc5.LAG).groupBy(fc5.SALESAREA,fc5.VERSION,fc5.WEEK,fc5.LAG).agg(f.sum('SQTY').alias('SQTY'),f.sum('DPFQTY').alias('DPFQTY'),f.sum('DSFQTY').alias('DSFQTY'))
fc8 = EUIDH_HORIZON_REPORT_AGGR.alias("fc8")
#DFToView(fc9,'fc9')
#.agg(f.sum('TOTAL_FCST').alias('TOTAL_FCSTDS'))


In [27]:
%sql
--select distinct DSFQTY from VW_EAN_DS_DP_AGG limit 10
select distinct DS_FRCST,LAG  FROM fc6 limit 10 --
--sp  FULL OUTER JOIN s    ON sp.ITEM = s.ITEM AND sp.VERSION = s.VERSIONS AND sp.SALESAREA = s.SalesArea AND sp.WEEK = s.SDATEFormat
--order by DPFQTY

DS_FRCST,LAG
46235.638999999996,LagW_01
1048.531,LagW_01
4872.374,LagW_01
1022.684,LagW_01
1771.7990000000002,LagW_01
35352.310000000005,LagW_01
1620.5570000000002,LagW_01
21662.757,LagW_01
2646.579,LagW_01
25439.142,LagW_01
